In [5]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [6]:
df1= pd.read_csv(r'D:\ML\Movies_recommender\TMDB\tmdb_5000_movies.csv')
df2=pd.read_csv(r'D:\ML\Movies_recommender\TMDB\tmdb_5000_credits.csv')

In [7]:
df=pd.merge(df1,df2,left_on = 'id', right_on = 'movie_id',how = 'left')

In [8]:
df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


## features that im gonna use
genres
id
keywwords
overview
release_date
title_x
cast
crew


In [9]:
df.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title_x                    0
vote_average               0
vote_count                 0
movie_id                   0
title_y                    0
cast                       0
crew                       0
dtype: int64

In [10]:
movies= df[['id','genres','keywords','overview','title_x','cast','crew']].rename(columns = {'title_x':'title'})

In [11]:
movies

,id,genres,keywords,overview,title,cast,crew
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...
4798,9367,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],A newlywed couple's honeymoon is upended by th...,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...","Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,[],[],When ambitious New York attorney Sam is sent t...,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [12]:
# dropping null rows as they are only 3 in number
movies.dropna(inplace = True)

In [13]:
movies.iloc[0]['genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [14]:
movies.iloc[0]['keywords']

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [15]:
movies.loc[movies['id']==25975]['crew']

4802    [{"credit_id": "58ce021b9251415a390165d9", "de...
Name: crew, dtype: object

In [16]:
#converting genres to list
movies['genres'] = movies['genres'].apply(lambda x : [i['name'] for i in ast.literal_eval(x)])

In [17]:
#converting keywords to list
movies['keywords'] = movies['keywords'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)])

In [18]:
movies['cast'] = movies['cast'].apply(lambda x : [i['name'] for i in ast.literal_eval(x)[:3]])

In [19]:
def fetch_director(x):
    l=[]
    for i in ast.literal_eval(x):
        if i['job']=="Director":
            l.append(i['name'])
            break
    return l

In [20]:
#movies['director'] = movies['crew'].apply(lambda x: next((i['name'] for i in ast.literal_eval(x) if i['job'] == 'Director'), ""))
#movies['director'] = movies['crew'].apply(lambda x: [i['name'] for i in ast.literal_eval(x) if i['job']=='Director'])
#using this as i wanted only one director and wanted to hande the case where there were no directors as i encountered in the first method
movies['director'] = movies['crew'].apply(fetch_director)

In [21]:
movies

,id,genres,keywords,overview,title,cast,crew,director
0,19995,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",[James Cameron]
1,285,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",[Gore Verbinski]
2,206647,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",[Sam Mendes]
3,49026,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",[Christopher Nolan]
4,49529,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",[Andrew Stanton]
...,...,...,...,...,...,...,...,...
4798,9367,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...",El Mariachi just wants to play his guitar and ...,El Mariachi,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",[Robert Rodriguez]
4799,72766,"[Comedy, Romance]",[],A newlywed couple's honeymoon is upended by th...,Newlyweds,"[Edward Burns, Kerry Bishé, Marsha Dietlein]","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",[Edward Burns]
4800,231617,"[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...","""Signed, Sealed, Delivered"" introduces a dedic...","Signed, Sealed, Delivered","[Eric Mabius, Kristin Booth, Crystal Lowe]","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...",[Scott Smith]
4801,126186,[],[],When ambitious New York attorney Sam is sent t...,Shanghai Calling,"[Daniel Henney, Eliza Coupe, Bill Paxton]","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",[Daniel Hsia]


In [22]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [23]:
def remove_space(df, cols):
    for col in cols:
        df[col] = df[col].apply(lambda x : [i.replace(" ","") for i in x])
    return df

In [24]:
# removing spaces from elements in the list of features like genres and crew so that long names and space seperated tags are not understood differently
# ['Science fiction','drama']:-> ['ScienceFiction','drama']

movies=remove_space(movies,['genres','keywords','cast','director'])

In [25]:
movies

,id,genres,keywords,overview,title,cast,crew,director
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",[JamesCameron]
1,285,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",[GoreVerbinski]
2,206647,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",[SamMendes]
3,49026,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman]","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",[ChristopherNolan]
4,49529,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",[AndrewStanton]
...,...,...,...,...,...,...,...,...
4798,9367,"[Action, Crime, Thriller]","[unitedstates–mexicobarrier, legs, arms, paper...","[El, Mariachi, just, wants, to, play, his, gui...",El Mariachi,"[CarlosGallardo, JaimedeHoyos, PeterMarquardt]","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",[RobertRodriguez]
4799,72766,"[Comedy, Romance]",[],"[A, newlywed, couple's, honeymoon, is, upended...",Newlyweds,"[EdwardBurns, KerryBishé, MarshaDietlein]","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",[EdwardBurns]
4800,231617,"[Comedy, Drama, Romance, TVMovie]","[date, loveatfirstsight, narration, investigat...","[""Signed,, Sealed,, Delivered"", introduces, a,...","Signed, Sealed, Delivered","[EricMabius, KristinBooth, CrystalLowe]","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...",[ScottSmith]
4801,126186,[],[],"[When, ambitious, New, York, attorney, Sam, is...",Shanghai Calling,"[DanielHenney, ElizaCoupe, BillPaxton]","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",[DanielHsia]


In [26]:
movies['tags'] = movies['genres'] + movies['genres']+movies['keywords']+movies['overview']+movies['cast']+movies['director']

In [27]:
df=movies[['id','title','tags']]

In [28]:
df['tags'] = df['tags'].apply(lambda x: ' '.join(x))

C:\Users\ravik\AppData\Local\Temp\ipykernel_8648\1753472850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: ' '.join(x))


In [29]:
df['tags'] = df['tags'].apply(lambda x : x.lower())

C:\Users\ravik\AppData\Local\Temp\ipykernel_8648\4152385510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x : x.lower())


In [30]:
# data={'a': [[1,2,3],[4,5]],
#      'b' : [[5,4],[6,7]]}

# tds= pd.DataFrame(data)
# tds

In [31]:
ps = PorterStemmer()

In [32]:
# This is to apply stemming on the data string say we have ['love','lovers',' lovely'] :-> ['love','love','love'] basically simalar words will be formed as one word so we don't pass those in the data
# stemming helper function
'''
input :  text
process :  convert string to list and apply stemming operation on it.
return :  string with stemming applied.
'''
def stemmer(text):
    lst = []
    
    for i in text.split():
        lst.append(ps.stem(i))
    return " ".join(lst)
    

In [33]:
df['tags'] = df['tags'].apply(stemmer)

C:\Users\ravik\AppData\Local\Temp\ipykernel_8648\1072922980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stemmer)


In [34]:
cv= CountVectorizer(stop_words = 'english', max_features=5000)

## Bag of words
in the cell above appplying bag of words technique to generate vectors for the tags
1. max_features is the top n most frequent words selected.
2. each word for the data line is then selected and checked if it is available in the selected words
3. if yes it's count is incremented and stored in the list 
4. each line items list from above step is appended into the list to make a vector

In [35]:
vec =  cv.fit_transform(df['tags']).toarray()

In [36]:
#print(cv.get_feature_names_out())

In [37]:
similarity_vector = cosine_similarity(vec)

In [38]:
def recommend(movie):
    movie_index = df[df['title']==movie].index[0]
    cos_dists = similarity_vector[movie_index]
    top_5_movies = sorted(enumerate(cos_dists), key = lambda x: x[1], reverse=True)[1:6]
    # the complete tupe is returned here.
    #return top_5_movies
    # to print the movie names
    print(top_5_movies)
    for i in top_5_movies:
        print(df.iloc[i[0]].title)
    

In [39]:
recommend("Batman Begins")

[(65, 0.48275419151237386), (4635, 0.4743719033001881), (3292, 0.4142967427397615), (3, 0.4075854416901342), (3818, 0.39900373444305315)]
The Dark Knight
Amidst the Devil's Wings
10th & Wolf
The Dark Knight Rises
Defendor
